In [21]:
!pip install agentpy

### Explorer agent ###

The following code simulates an explorer that crosses an environment with cells representing road, grass, water, and mountain. Address the following tasks:


1.   Propose a solution to minimize the total travel cost with the set values.
2.   Modify the solution to simulate an agent that prefers walking through grass than on the road.
3.   Modify the solution to simulate an agent that prefers climbing mountains than swimming.



In [24]:
import agentpy as ap
import numpy as np
import random, json
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns, IPython
from matplotlib import pyplot as plt, cm
import heapq


# Define movement directions (Up, Down, Left, Right)
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

#  Heuristic function: Manhattan distance
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

#  A* Algorithm


class MazeAgent(ap.Agent):
    '''
    Initializing agent elements:
    - Has a route and starts with next step
    '''
    def setup(self):

        self.env = self.model.env
        self.route = self.env.a_star_search(self.env.environment, self.p.start, self.p.goal)
        self.next_step = 1

    '''
    Actual action execution. d
    '''
    def execute(self):
      next_pos = self.route[self.next_step]
      self.model.env.move_to(self, next_pos)
      self.next_step += 1

    def get_position(self):
        return self.env.positions[self]



class Maze(ap.Grid):
    def setup(self):
        # Initialize the maze environment
        self.environment = np.copy(self.p.maze)
        self.environment[self.p.goal] = self.p.goal_value

    '''
    A* Algorithm. Calculates the shortest path from start to goal.
    '''
    def a_star_search(self, grid, start, goal):
      rows, cols = grid.shape
      open_set = []
      heapq.heappush(open_set, (0, start))  # (cost, position)

      came_from = {}  # Track the path
      g_score = {start: 0}  # Cost from start to each node
      f_score = {start: heuristic(start, goal)}  # Estimated total cost

      while open_set:
          _, current = heapq.heappop(open_set)

          if current == goal:
              #  Reconstruct and return the path
              path = []
              while current in came_from:
                  path.append(current)
                  current = came_from[current]
              path.append(start)
              return path[::-1]  # Reverse the path

          for dx, dy in DIRECTIONS:
              neighbor = (current[0] + dx, current[1] + dy)

              #  Check boundaries
              if not (0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols):
                  continue

              #  Check if the cell is impassable
              if grid[neighbor] in [-1, -10]:
                  continue

              #  Calculate new cost
              tentative_g_score = g_score[current] + grid[neighbor]
              if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                  came_from[neighbor] = current
                  g_score[neighbor] = tentative_g_score
                  f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                  heapq.heappush(open_set, (f_score[neighbor], neighbor))

      return None

'''
Maze model. definition of class MazeModel.
'''
class MazeModel(ap.Model):
    def setup(self):
        self.env = Maze(self, shape=maze.shape)
        self.agent = MazeAgent(self)
        self.env.add_agents([self.agent], positions=[self.p.start])


    def step(self):
        self.agent.execute()

    def update(self):
        if self.agent.get_position() == self.model.p.goal:
            print('ending')
            self.stop()

    # Report found route and Q-values
    def end(self):
        self.report('Q-Table', self.agent.Q)




def animation_plot(model, ax):
    N, M = model.p.maze.shape
    grid = np.copy(maze)
    grid[model.p.goal] = goal
    agent = list(model.env.agents)[0]
    state = model.env.positions[agent]
    grid[state] = explorer

    # Colors: black = edge, white = floor, green = goal, blue = agent
    color_dict = {
    1: "#4CAF50",   # Green - Street in good condition
    2: "#A1887F",   # Brown - Dirt road
    4: "#FFEB3B",   # Yellow - Street with cracks and water leaks
    5: "#FF5722",   # Orange-Red - Potholes
    -1: "#000000",  # Black - Road closures (impassable)
    -10: "#7B1FA2", # Purple - Buildings (impassable)
    goal: "#2196F3",  # Blue - Goal (Destination)
    explorer: "#E91E63"  # Pink - Explorer (Agent)
}


    ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)

explorer, goal = -3, -2

maze = np.load("streets-1.npy")

parameters = {
    'maze': maze,
    'start': (5,3),
    'goal': (16, 26),
    'goal_value': 10000,
    'steps': 150,
}

# Environment representation with a grid




fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
mazeModel = MazeModel(parameters)
animation = ap.animate(mazeModel, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

ending
